In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import tensorflow as tf
from tensorflow import keras
import cv2
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

tf.random.set_seed(122)
tf.config.experimental.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
input_shape = (256, 256, 3)
image_shape = (256, 256)
num_classes = 5
batch_size = 10

train_data, val_data = keras.utils.image_dataset_from_directory('flower_images', batch_size=batch_size, image_size=image_shape, label_mode='categorical', seed=225, validation_split= 0.2, subset='both')

Found 4999 files belonging to 5 classes.
Using 4000 files for training.
Using 999 files for validation.


In [3]:
data_aug = keras.Sequential([
    keras.layers.Rescaling(scale=1./127.5, offset=-1),
    keras.layers.RandomFlip(),
    keras.layers.RandomRotation(0.35)
])

In [4]:
def ConvMixer(dim: int, patch_size: int, depth: int, kernel_size: int, num_classes: int):
    input = keras.Input(shape=input_shape)
    
    input = data_aug(input)
    
    x = keras.layers.Conv2D(filters=dim, kernel_size=patch_size, strides=patch_size, activation='gelu')(input)
    x1 = keras.layers.BatchNormalization()(x)
    
    for i in range(depth):
        x = keras.layers.Conv2D(filters=dim, kernel_size=kernel_size, padding='same', groups=dim, activation='gelu')(x1)
        x = keras.layers.BatchNormalization()(x)
        x = keras.layers.add([x1, x])
        
        x = keras.layers.Conv2D(filters=dim, kernel_size=1, activation='gelu')(x)
        
        x1 = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.AveragePooling2D(pool_size=(1, 1))(x1)
    x = keras.layers.Flatten()(x)
    x = keras.layers.Dropout(0.2)(x)
    output = keras.layers.Dense(units=num_classes, activation='softmax')(x)
    
    
    return keras.Model(input, output)



model = ConvMixer(dim=256, patch_size=7, kernel_size=9, depth=4, num_classes=5)
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 36, 36, 256)  37888       ['input_2[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 36, 36, 256)  1024       ['conv2d[1][0]']                 
 alization)                                                                                       
                                                                                              

In [5]:
callbacks = [
    keras.callbacks.TensorBoard(), 
    keras.callbacks.ReduceLROnPlateau(patience=5, factor=0.1),
    keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=15),
    keras.callbacks.ModelCheckpoint(filepath='checkpoints/convmixer-256-4-{epoch}.h5', save_best_only=True, save_weights_only=True, monitor='val_accuracy'),
    ]

In [ ]:
model.compile(
    loss=keras.losses.CategoricalCrossentropy(), 
    optimizer=keras.optimizers.Adam(learning_rate=0.001), 
    metrics='accuracy',
    )

history = model.fit(train_data, epochs=50, batch_size=batch_size, validation_data=val_data, callbacks=callbacks, verbose='auto')

In [ ]:
ax = sns.lineplot(data={'loss': history.history['loss'], 'val_loss': history.history['val_loss']})
ax.set(xlabel='Epoch', ylabel='loss')

In [ ]:
ax = sns.lineplot(data={'accuracy': history.history['accuracy'], 'val_accuracy': history.history['val_accuracy']})
ax.set(xlabel='epoch', ylabel='accuracy', ylim=(0,1))